In [1]:
import pandas as pd
df = pd.read_csv('E:/NTHUcourses/三上/機器學習/final/dataset/train.csv', header = None)

Dataset:
https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/discussion/41806#latest-666617

In [2]:
import numpy as np
pixel = df[1].values
label = df[0].values
X = pixel[1:]
y = label[1:]
for i in range(0, len(X)):
    X[i] = X[i].split(" ")
    y[i] = int(y[i])
    for j in range(0, len(X[i])):
        X[i][j] = float(X[i][j])/255.0
for i in range(0, len(X)):
    X[i]=np.array(X[i])
X = np.vstack(X)
y = np.vstack(y)

將Label轉成onehot的形式，並且在輸入的部分用reshape將X轉成(28709, 48, 48, 1)的四維陣列

In [3]:
from sklearn.preprocessing import label_binarize
y_onehot = label_binarize(y, [0, 1, 2, 3, 4, 5, 6])
X=X.reshape(28709, 48, 48, 1)

切出30%的資料做為測試用

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y_onehot,
                    test_size = 0.3,
                    random_state = 0,
                    stratify = y)

用keras來建構我的神經網路

首先是connvolution layer，因為圖片是48X48的大小，並沒有太大，所以filter的大小選擇用3X3，並用same padding保持圖片大小
接著加一層pooling layer，採用2X2的大小將圖片縮減為24X24

再重複兩次上面的過程，並加一層dropout layer降低overfit的效應

接著加一層hidden layer，因為最後的output是7種class，經過嘗試後選擇用7X16=224個神經元作為hidden layer的大小

最後是7個神經元的output layer
(因為features的值都是介於0-1之間，所以activation都是用relu就好)

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(filters = 20,
                 kernel_size = (3,3),
                 padding = 'same',
                 input_shape = (48,48,1),
                 activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2),))
model.add(Conv2D(filters = 20,
                 kernel_size = (3,3),
                 padding = 'same',
                 activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(filters = 20,
                 kernel_size = (3,3),
                 padding = 'same',
                 activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(units = 224,
                activation = 'relu'))
model.add(Dense(units = 7, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

試了幾次之後發現這次資料收斂很快，如果batch size太大的話雖然每個epoch時間變短，但需要用很多個epoch才能達到較高的準確度，並且會有overfit的現象。所以最後挑選batch size為64，並切出20%的資料作驗證

其實過了10個epoch之後也出現了overfit的狀況，不過還是有稍微提升了點準確度。最後的結果為52.67%的準確度，雖然並沒有很高，但參考網站上的排行榜也已經高於一半的參賽者。

這次的準確度不足，我猜想可能是圖片的大小並不大，只有48X48的尺寸，可能在辨識情緒上有點不足；也可能是我在神經網路的設計上仍有缺陷，或是應該使用其他模型去完成分類

In [8]:
train_history = model.fit(x=X_train, y=y_train,
                          validation_split = 0.2,
                          epochs = 15, batch_size = 64, verbose = 2)
scores = model.evaluate(X_test, y_test)
print("Accuracy: %f" %scores[1])

Train on 16076 samples, validate on 4020 samples
Epoch 1/15
 - 46s - loss: 1.7594 - acc: 0.2822 - val_loss: 1.6550 - val_acc: 0.3351
Epoch 2/15
 - 48s - loss: 1.6014 - acc: 0.3778 - val_loss: 1.5274 - val_acc: 0.4127
Epoch 3/15
 - 46s - loss: 1.5099 - acc: 0.4183 - val_loss: 1.4572 - val_acc: 0.4430
Epoch 4/15
 - 45s - loss: 1.4524 - acc: 0.4439 - val_loss: 1.4301 - val_acc: 0.4597
Epoch 5/15
 - 38s - loss: 1.3980 - acc: 0.4656 - val_loss: 1.3686 - val_acc: 0.4799
Epoch 6/15
 - 34s - loss: 1.3570 - acc: 0.4796 - val_loss: 1.4009 - val_acc: 0.4600
Epoch 7/15
 - 37s - loss: 1.3184 - acc: 0.4974 - val_loss: 1.3295 - val_acc: 0.4863
Epoch 8/15
 - 37s - loss: 1.2741 - acc: 0.5167 - val_loss: 1.3045 - val_acc: 0.5047
Epoch 9/15
 - 46s - loss: 1.2407 - acc: 0.5282 - val_loss: 1.3122 - val_acc: 0.5042
Epoch 10/15
 - 39s - loss: 1.2036 - acc: 0.5445 - val_loss: 1.2855 - val_acc: 0.5172
Epoch 11/15
 - 36s - loss: 1.1665 - acc: 0.5624 - val_loss: 1.2837 - val_acc: 0.5047
Epoch 12/15
 - 34s - loss